In [1]:
from OFDM import *
%matplotlib inline

In [2]:
known_seq_dec = np.genfromtxt("handouts/week_2_challenge/a7r56tu_knownseq.csv", delimiter=r"\s+")

known_seq = np.array([])

for b in known_seq_dec:
    x = format(int(b),'02b')
    for i in x:
        known_seq = np.append(known_seq, int(i))

# Question 1

In [3]:
rx = receiver(ofdm_symbol_size= 4096, cp_length=0, modulation="QPSK", fs=48000, end_chirp=False)
rx.gap_length = 4096

In [4]:
fs, signal = wavfile.read("handouts/week_2_challenge/a7r56tu_received.wav")
rx.fs = fs
signal = signal / 30000

In [5]:
rx_bits = rx.receive(signal)

------------------------------------------
Receive 
------------------------------------------
OFDM Paramters:
Number of actual Sub Carriers:      2047 
Cyclic prefix length:               0 
Modulation method:                  QPSK
Number of received OFDM symbols:    2894
Number of received bits:            11848036


In [6]:
save_file(rx_bits)

File Name: a7r56tu.gif
File Size: 1476376


# Question 2